In [ ]:
import os
from os.path import join
import sys
import json
import pandas as pd
import numpy as np
np.warnings.filterwarnings('ignore')
import traceback
import shutil

import random
from slugify import slugify
from collections import Counter
from itertools import combinations

from pprint import pprint

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path: sys.path.append(nb_dir)


import altair as alt
from altair import datum
from IPython.display import display

# for the notebook only (not for JupyterLab) run this command once per session
alt.renderers.enable('default')

In [2]:
general_type_to_dtype = {
    'c': str,
    'q': np.float64
}

In [3]:
output_dir = '../experiment_data'
os.makedirs(output_dir, exist_ok=True)

In [4]:
raw_specs = [{"description": {"id": 0}, "$schema": "https://vega.github.io/schema/vega-lite/v3.0.0-rc5.json", "mark": "point", "encoding": {"x": {"field": "Q1", "type": "quantitative"}, "y": {"field": "Q2", "type": "quantitative"}, "color": {"field": "name", "type": "nominal"}}}, {"description": {"id": 1}, "$schema": "https://vega.github.io/schema/vega-lite/v3.0.0-rc5.json", "mark": "point", "encoding": {"x": {"field": "Q1", "type": "quantitative"}, "y": {"field": "Q2", "type": "quantitative"}, "row": {"field": "name", "type": "nominal"}}}, {"description": {"id": 2}, "$schema": "https://vega.github.io/schema/vega-lite/v3.0.0-rc5.json", "mark": "point", "encoding": {"x": {"field": "Q1", "type": "quantitative"}, "color": {"field": "Q2", "type": "quantitative"}, "y": {"field": "name", "type": "nominal"}}}, {"description": {"id": 3}, "$schema": "https://vega.github.io/schema/vega-lite/v3.0.0-rc5.json", "mark": "point", "encoding": {"x": {"field": "Q1", "type": "quantitative"}, "size": {"field": "Q2", "type": "quantitative", "scale": {"range": [1, 400]}}, "y": {"field": "name", "type": "nominal"}}}, {"description": {"id": 4}, "$schema": "https://vega.github.io/schema/vega-lite/v3.0.0-rc5.json", "mark": "point", "encoding": {"color": {"field": "Q1", "type": "quantitative"}, "x": {"field": "Q2", "type": "quantitative"}, "y": {"field": "name", "type": "nominal"}}}, {"description": {"id": 5}, "$schema": "https://vega.github.io/schema/vega-lite/v3.0.0-rc5.json", "mark": "point", "encoding": {"size": {"field": "Q1", "type": "quantitative", "scale": {"range": [1, 400]}}, "x": {"field": "Q2", "type": "quantitative"}, "y": {"field": "name", "type": "nominal"}}}, {"description": {"id": 6}, "$schema": "https://vega.github.io/schema/vega-lite/v3.0.0-rc5.json", "mark": "point", "encoding": {"y": {"field": "Q1", "type": "quantitative"}, "x": {"field": "Q2", "type": "quantitative"}, "color": {"field": "name", "type": "nominal"}}}, {"description": {"id": 7}, "$schema": "https://vega.github.io/schema/vega-lite/v3.0.0-rc5.json", "mark": "point", "encoding": {"y": {"field": "Q1", "type": "quantitative"}, "x": {"field": "Q2", "type": "quantitative"}, "row": {"field": "name", "type": "nominal"}}}, {"description": {"id": 8}, "$schema": "https://vega.github.io/schema/vega-lite/v3.0.0-rc5.json", "mark": "point", "encoding": {"y": {"field": "Q1", "type": "quantitative"}, "color": {"field": "Q2", "type": "quantitative"}, "x": {"field": "name", "type": "nominal"}}}, {"description": {"id": 9}, "$schema": "https://vega.github.io/schema/vega-lite/v3.0.0-rc5.json", "mark": "point", "encoding": {"y": {"field": "Q1", "type": "quantitative"}, "size": {"field": "Q2", "type": "quantitative", "scale": {"range": [1, 400]}}, "x": {"field": "name", "type": "nominal"}}}, {"description": {"id": 10}, "$schema": "https://vega.github.io/schema/vega-lite/v3.0.0-rc5.json", "mark": "point", "encoding": {"color": {"field": "Q1", "type": "quantitative"}, "y": {"field": "Q2", "type": "quantitative"}, "x": {"field": "name", "type": "nominal"}}}, {"description": {"id": 11}, "$schema": "https://vega.github.io/schema/vega-lite/v3.0.0-rc5.json", "mark": "point", "encoding": {"size": {"field": "Q1", "type": "quantitative", "scale": {"range": [1, 400]}}, "y": {"field": "Q2", "type": "quantitative"}, "x": {"field": "name", "type": "nominal"}}}]
json.dump(raw_specs, open(join(output_dir, 'specs.json'), 'w'))

In [5]:
'''
encoding_type = 'x', 'y', 'color', 'size'
field_abbreviation = 'c', 'q1', 'q2'
abbreviation_to_field_name = {  c: c_field_name, q1: q1_field_name, q2: q2_field_name}
'''
def get_encoding_expansion(encoding_type, field_abbreviation, abbreviation_to_field_name):
    result = { 
        'field': abbreviation_to_field_name[field_abbreviation],
        'type': 'quantitative' if field_abbreviation in [ 'q1', 'q2' ] else 'nominal'
    }
    if encoding_type == 'size':
        result['scale'] = { 'range': [1, 400] }
    return result

## Functions

In [6]:
def generate_question_find_value(df, abbreviation_to_field_name):
    c_name = abbreviation_to_field_name['c']
    q1_name, q2_name = abbreviation_to_field_name['q1'], abbreviation_to_field_name['q2']
    
    # Filter out cases where both Q points are null
    both_q_null = df[[ q1_name, q2_name ]].isnull().any(axis=1)
    df = df[~both_q_null]

    q1_values = df[q1_name]
    q1_max = np.nanmax(q1_values)
    q1_min = np.nanmin(q1_values)
    q1_range = np.abs(q1_max - q1_min)

    point = df.sample(1)
    correct_value = point[q1_name].item()
    
    mid_point = (q1_max + q1_min) / 2
    mid_range = (q1_max - q1_min) / 2

    if correct_value > mid_point:
        incorrect_value = correct_value - (mid_range).item()
    else: 
        incorrect_value = correct_value + (mid_range).item()
    
    if correct_value == incorrect_value:
        return None
    
    # incorrect_value = (correct_value - q1_range / 2).item() if (random.random() < 0.5) else (correct_value + q1_range).item()
    question_text = {
        'text': 'What is the <b>{}</b> value of the data point A?'.format(q1_name),
        'structure': [
            { 'type': 'text', 'value': 'What is the' },
            { 'type': 'field', 'value': q1_name },
            { 'type': 'text', 'value': 'of the data point A?' }
        ]
    }
    
    # Randomize options
    options = [ correct_value, incorrect_value ]
    correct = 0
    if random.random() < 0.5:
        options = [ incorrect_value, correct_value ]
        correct = 1
    
    annotation = {
        'annotated': 'A'
    }
    annotation[c_name] = point[c_name].item()
    annotation[q1_name] = point[q1_name].item()
    annotation[q2_name] = point[q2_name].item()
    annotations = [ annotation ]

    question = {
        'q_field': q1_name,
        'q1': q1_name,
        'q2': q2_name,
        'c': c_name,
        'question': question_text,
        'options': options,
        'correct': correct,
        'task': 'find_value',
        'annotated': annotations
    }
    return question

In [7]:
def generate_question_compare_values(df, abbreviation_to_field_name):
    c_name, q1_name, q2_name = abbreviation_to_field_name['c'], abbreviation_to_field_name['q1'], abbreviation_to_field_name['q2']

    # Randomly decide if A and B are from same or different group
    sample_same_group = random.choice([ True, False ])
    groups = list(df[c_name].unique())
    if sample_same_group: group_a = group_b = random.sample(groups, 1)[0]
    else: group_a, group_b = random.sample(groups, 2)
    
    # Get Range
    q1_values = df[q1_name]
    q1_max = np.nanmax(q1_values)
    q1_min = np.nanmin(q1_values)
    q1_range = np.abs(q1_max - q1_min)
    q1_mid_range = q1_range / 2
    q1_mid_point = (q1_max + q1_min) / 2

    # Select Point A
    point_a = df[df[c_name] == group_a].sample(1)
    point_a_q1 = point_a[q1_name].item()

    # Select Point B
    if point_a_q1 < q1_mid_point:
        ideal_point_b_q1 = point_a_q1 + q1_mid_range
    else:
        ideal_point_b_q1 = point_a_q1 - q1_mid_range
    
    identical_points = (df[c_name] == point_a[c_name].item()) & (df[q1_name] == point_a[q1_name].item()) & (df[q2_name] == point_a[q2_name].item())
    df_excluding_point_a = df[~identical_points]
    df_subset = df_excluding_point_a[df_excluding_point_a[c_name] == group_b]
    
    point_b_candidate_distances = np.abs(df_subset[q1_name] - ideal_point_b_q1)
    point_b = pd.DataFrame(df.iloc[point_b_candidate_distances.idxmin(), :]).transpose()
    point_b_q1 = point_b[q1_name].item()
    
    more_or_less = random.choice(['higher', 'lower'])
    if more_or_less == 'more': correct = 0 if (point_a_q1 > point_b_q1) else 1
    else: correct = 0 if (point_a_q1 < point_b_q1) else 1
            
    question_text = {
        'text': 'Which data point has a <u>{}</u> value of <b>{}</b>?'.format(more_or_less, q1_name),
        'structure': [
            { 'type': 'text', 'value': 'Which data point has' },
            { 'type': 'operator', 'value': more_or_less },
            { 'type': 'field', 'value': q1_name }
        ]  
    }
    
    annotations = []
    
    if (point_a[q1_name].item() == point_b[q1_name].item()) and \
        (point_a[q2_name].item() == point_b[q2_name].item()) and \
        (point_a[c_name].item() == point_b[c_name].item()):
        print('Point A and Point B are the same')
        print(point_a, point_b)
        return None
    
    for point, point_name in [ (point_a, 'A'), (point_b, 'B') ]:
        annotation = { 'annotated': point_name }
        annotation[c_name] = point[c_name].item() # annotation_c_name
        annotation[q1_name] = point[q1_name].item()
        annotation[q2_name] = point[q2_name].item()
        
        if any(pd.isnull([ point[c_name].item(), point[q1_name].item(), point[q2_name].item() ])):
            print(point[c_name].item(), point[q1_name].item(), point[q2_name].item())
            return None
            
        annotations.append(annotation)

    question = {
        'q_field': q1_name,
        'q1': q1_name,
        'q2': q2_name,
        'c': c_name,
        'question': question_text,
        'options': [ 'A', 'B' ],
        'correct': correct,
        'task': 'compare_values',
        'annotated': annotations,
        'metadata': {
            'same_group': sample_same_group
        }
    }
    return question

In [8]:
def generate_question_find_maximum(df, abbreviation_to_field_name):
    c_name, q1_name, q2_name = abbreviation_to_field_name['c'], abbreviation_to_field_name['q1'], abbreviation_to_field_name['q2']

    # Find category of point containing highest q1 (M)
    q1_values = df[q1_name]
    row_with_highest_q1_value = df.iloc[[q1_values.idxmax()]]
    highest_q1_value = row_with_highest_q1_value[q1_name].item()
    category_with_highest_q1_value = row_with_highest_q1_value[c_name].item()
    
    # Find category with highest q1 value closest to M - range(q1) / 2
    range_q1 = np.abs(np.nanmax(q1_values) - np.nanmin(q1_values))
    target_second_category_value = np.nanmax(q1_values) - (range_q1 / 2)
    df_max_by_category = df[df[c_name] != category_with_highest_q1_value].groupby([ c_name ]).max()
    df_sorted_distance_from_target = df_max_by_category.iloc[(df_max_by_category[q1_name] - target_second_category_value).abs().argsort()[:1]]
    category_with_closest_to_target_q1_value = df_sorted_distance_from_target.index.values[0]
    
    # Populate options
    random_choice = random.choice([0, 1])  # Which option is correct
    correct = random_choice
    reverse = -1 if random_choice else 1
    options = [ category_with_highest_q1_value, category_with_closest_to_target_q1_value ][::reverse]

    # Generate question test
    question_text = {
        'text': 'Which value of <b>{}</b> contains the data point with the highest value of <b>{}</b>?'.format(c_name, q1_name),
        'structure': [
            { 'type': 'text', 'value': 'Which' },
            { 'type': 'field', 'value': c_name },
            { 'type': 'text', 'value': 'has the data point with the highest' },
            { 'type': 'field', 'value': q1_name }
        ]  
    }

    # Return question
    question = {
        'q_field': q1_name,
        'q1': q1_name,
        'q2': q2_name,
        'c': c_name,
        'question': question_text,
        'options': options,
        'correct': correct,
        'task': 'compare_values',
        'annotated': []
    }
    return question

In [9]:
def generate_question_compare_averages(df, abbreviation_to_field_name):
    c_name, q1_name, q2_name = abbreviation_to_field_name['c'], abbreviation_to_field_name['q1'], abbreviation_to_field_name['q2']

    # Find category averages and target_difference
    q1_values = df[q1_name]
    range_q1 = np.abs(np.nanmax(q1_values) - np.nanmin(q1_values))
    df_average_by_category = df.groupby([ c_name ]).mean()
    target_difference_of_averages = 0.3 * range_q1

    # Calculate difference of means of each group
    differences_by_pair = []
    groups = list(df[c_name].unique())
    for ( group_1, group_2 ) in combinations(groups, 2):
        group_1_q1 = df_average_by_category.loc[group_1][q1_name]
        group_2_q1 = df_average_by_category.loc[group_2][q1_name]
        difference = np.abs(group_1_q1 - group_2_q1)
    
        differences_by_pair.append({
            'group_1': group_1,
            'group_2': group_2,
            'difference': difference
        })
        
    # Get closest match
    df_differences_by_pair = pd.DataFrame(differences_by_pair)
    closest_matching_pair = df_differences_by_pair.iloc[(df_differences_by_pair['difference'] - target_difference_of_averages).abs().argsort()[:1]]
    
    # Get means of final groups
    group_1, group_2 = closest_matching_pair['group_1'], closest_matching_pair['group_2']  # Accessor indices
    mean_group_1, mean_group_2 = df_average_by_category.loc[group_1][q1_name][0], df_average_by_category.loc[group_2][q1_name][0]
    group_1, group_2 = group_1.item(), group_2.item()  # Actual values
    
    # Populate options
    random_choice = random.choice([0, 1])  # Which option is correct
    correct = 0 if ( mean_group_1 > mean_group_2 ) else 1
    options = [ group_1, group_2 ]

    # Generate question test
    question_text = {
        'text': 'Considering all data points for <b>{}</b>, which of the two following values of <b>{}</b> has the greater <u>average</u> <b>{}</b>?'.format(c_name, c_name, q1_name),
        'structure': [
            { 'type': 'text', 'value': 'Considering all data points for' },
            { 'type': 'field', 'value': '{},'.format(c_name) },
            { 'type': 'text', 'value': ', which of the two following values of' },
            { 'type': 'field', 'value': c_name },
            { 'type': 'text', 'value': 'has the greater' },
            { 'type': 'operator', 'value': 'average' },
            { 'type': 'field', 'value': q1_name }
        ]  
    }

    # Return question
    question = {
        'q_field': q1_name,
        'q1': q1_name,
        'q2': q2_name,
        'c': c_name,
        'question': question_text,
        'options': options,
        'correct': correct,
        'task': 'compare_values',
        'annotated': []
    }
    return question

In [10]:
def detect_outlier_one_sided_MAD(points, thresh=3.5):
    """
    From: https://stackoverflow.com/questions/22354094/pythonic-way-of-detecting-outliers-in-one-dimensional-observation-data
    
    Returns a boolean array with True if points are outliers and False 
    otherwise.

    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.

    Returns:
    --------
        mask : A numobservations-length boolean array.

    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [11]:
def generate_question_detect_outliers(df, abbreviation_to_field_name, dim=1):
    c_name, q1_name, q2_name = abbreviation_to_field_name['c'], abbreviation_to_field_name['q1'], abbreviation_to_field_name['q2']

    if dim == 1:
        q_points = np.array(list(df[q1_name]))
    elif dim == 2:
        q_points = np.array(list(zip(df[q1_name], df[q2_name])))
    outliers = detect_outlier_one_sided_MAD(q_points)
    num_outliers = len([ o for o in outliers if o ])
    percent_outliers = num_outliers / df.shape[0]
    
    # Populate options
    options = [ 'Yes', 'No' ]
    correct = 0 if num_outliers else 1
    
    # Generate question test
    if dim == 1:
        question_text = {
            'text': 'Are there any outliers in <b>{}</b>?'.format(q1_name),
            'structure': [
                { 'type': 'text', 'value': 'Are there any outliers in' },
                { 'type': 'field', 'value': q1_name }
            ]  
        }
    elif dim == 2:
        question_text = {
            'text': 'Are there exceptions to the relationship between <b>{}</b> and <b>{}</b>?'.format(q1_name, q2_name),
            'structure': [
                { 'type': 'text', 'value': 'Are there exceptions to the relationship between' },
                { 'type': 'field', 'value': q1_name },
                { 'type': 'text', 'value': 'and' },
                { 'type': 'field', 'value': q2_name }
            ]  
        }

    # Return question
    question = {
        'q_field': q1_name,
        'q1': q1_name,
        'q2': q2_name,
        'c': c_name,
        'metadata': {
          'percent_outliers': percent_outliers  
        },
        'question': question_text,
        'options': options,
        'correct': correct,
        'task': 'compare_values',
        'annotated': []
    }
    
    return question

## Question Generation

In [20]:
# Input
corpuses = [ 'plotly', 'manyeyes', 'webtables', 'opendata' ]
input_data_dir = '../randomly_selected_cqq_specs/'
input_file_template = '{}_cqq_specs_with_data_all.tsv'

In [13]:
def make_serializable(obj):
    if isinstance(obj, dict):
        return { k: make_serializable(v) for (k, v) in obj.items() }
    elif isinstance(obj, list) or isinstance(obj, tuple) or isinstance(obj, np.ndarray):
        return [ make_serializable(d) for d in obj ]
    elif isinstance(obj, (int, np.integer)):
        return int(obj)
    elif isinstance(obj, (float, np.float)):
        return float(obj)
    else:
        return obj

In [14]:
def generate_question(df, abbreviation_to_field_name, corpus, dataset_id, original_dataset_id, spec_id, task_type):
    question = task_type_to_function[task_type](df, abbreviation_to_field_name)

    if not question:
        return None

    question = make_serializable(question)
    can_jsonify = json.loads(json.dumps(question))

    # Augment
    d = dict(question)
    d['cardinality'] = len(list(df[abbreviation_to_field_name['c']].unique()))
    d['corpus'] = corpus
    d['dataset_id'] = '{}.json'.format(dataset_id)
    d['original_dataset_id'] = original_dataset_id
    d['spec_id'] = spec_id
    return d

In [ ]:
max_per_source_dataset = 10

def get_sample_from_path(path, sample=3000, row_count=None, header=True, random_sample=True, columns=[]):
    df = pd.read_csv(
        path,
        sep='\t',
        nrows=10000000
    )
    df.columns = columns
    df.drop_duplicates(subset=['dataset_id', 'combination_number'], inplace=True)  # Drop dups
    df = df.groupby('dataset_id').head(max_per_source_dataset).reset_index(drop=True)   # Cap number per dataset
    df = df.sample(frac=1).reset_index(drop=True)  # Randomize
    # count_per_dataset_id = dict(df.groupby('dataset_id').count())
    # Max out number of datasets per source dataset
    
#    dataset_ids = np.unique(list(df['dataset_id']))   
    count_per_dataset_id = dict(df.groupby('dataset_id').count())
    # print(df.head())
    df = df.iloc[:sample, :]
    print(df.shape)
    return df

# Container for sampled datasets
sample_datasets_all_corpuses = pd.DataFrame(columns=['corpus', 'locator', 'dataset_id', 'combination_number', 'column_metadata', 'data'])
for corpus in corpuses:
    print('Corpus:', corpus)
    input_file_name = input_file_template.format(corpus)
    input_file_path = join(input_data_dir, input_file_name)
    #corpus_datasets_df = get_sample_from_path(input_file_path, sample=100000, columns=['corpus', 'locator', 'dataset_id', 'combination_number', 'column_metadata', 'data'])
    corpus_datasets_df = get_sample_from_path(input_file_path, sample=6000, columns=['corpus', 'locator', 'dataset_id', 'combination_number', 'column_metadata', 'data'])
    
    corpus_datasets_df = corpus_datasets_df.sample(frac=1).reset_index(drop=True)
    sample_datasets_all_corpuses = sample_datasets_all_corpuses.append(corpus_datasets_df, ignore_index=True)
    
sample_datasets_all_corpuses = sample_datasets_all_corpuses.sample(frac=1).reset_index(drop=True)
print(sample_datasets_all_corpuses.shape)

In [19]:
def format_column_and_group_names(df, abbreviation_to_field_name):
    df.rename(index=str, columns={
        abbreviation_to_field_name['c']: 'C',
        abbreviation_to_field_name['q1']: 'W',
        abbreviation_to_field_name['q2']: 'Z',
    }, inplace=True)
  
    replacement_dict = {}
    unique_groups = np.unique(df['C'])
    for i, g in enumerate(unique_groups):
        replacement_dict[g] = 'Group {}'.format(i)
    df.replace(replacement_dict, inplace=True)
    
    abbreviation_to_field_name['c'] = 'C'
    abbreviation_to_field_name['q1'] = 'W'
    abbreviation_to_field_name['q2'] = 'Z'
    df.reset_index(drop=True, inplace=True)
    return df, abbreviation_to_field_name

In [ ]:
task_type_to_function = {
    'read_value': generate_question_find_value,
    'compare_values': generate_question_compare_values,
    'find_maximum': generate_question_find_maximum,
    'compare_averages': generate_question_compare_averages,
    'detect_outliers': generate_question_detect_outliers
}
task_types = [ k for k in task_type_to_function.keys() ]

data_dir = join(output_dir, 'data')
shutil.rmtree(data_dir, ignore_errors=True)
os.makedirs(data_dir, exist_ok=True)

# Preserve datasets for later feature_extraction
valid_datasets = pd.DataFrame(columns=['corpus', 'locator', 'dataset_id', 'combination_number', 'column_metadata', 'data'])

total = 0
valid = 0
valid_per_corpus = Counter()

limit_per_corpus = 3000
print_every = 100
task_to_questions = { k: [] for k in task_type_to_function.keys() }
character_limit = 200
num_tasks = len(task_types)
num_specs = len(raw_specs)
spec_ids = [ v['description']['id'] for v in raw_specs ]
max_per_dataset_id = 10

# Deterministically populate v, t
num_per_v_t = 250 
expected_number_per_corpus_per_v_t = num_per_v_t / len(corpuses)
count_per_v_t = { s: { t: 0 for t in range(0, num_specs) } for s in spec_ids }
per_corpus_per_v_t = Counter()
current_spec_id = 0
current_task_id = 0

for (i, row) in sample_datasets_all_corpuses.iterrows():
    corpus = row['corpus']
    if per_corpus_per_v_t[corpus] >= expected_number_per_corpus_per_v_t: continue
    total += 1

    original_dataset_id = row['dataset_id']
    combination_number = row['combination_number']
    safe_dataset_id = slugify(original_dataset_id)
    safe_dataset_id = safe_dataset_id[:min(len(safe_dataset_id), character_limit)]
    dataset_id = '{}__{}'.format(safe_dataset_id, combination_number)
    column_metadata = json.loads(row['column_metadata'])
    
    try:
        data = row['data']
        df = pd.read_json(
            data,
            dtype={ cm['name']: np.dtype(cm['dtype']) for cm in column_metadata }
        )
        df = df[[ cm['name'] for cm in column_metadata ]]
    except Exception as e:
        print('Error reading data')
        continue
    
    abbreviation_to_field_name = {
        'c': [ cm['name'] for cm in column_metadata if (cm['general_type'] == 'c')][0],
        'q1': [ cm['name'] for cm in column_metadata if (cm['general_type'] == 'q')][0],
        'q2': [ cm['name'] for cm in column_metadata if (cm['general_type'] == 'q')][1]
    }
        
    df, abbreviation_to_field_name = format_column_and_group_names(df, abbreviation_to_field_name)
    spec = raw_specs[current_spec_id]
    spec_id = spec['description']['id']
    task_type = task_types[current_task_id]

    try:
        question = generate_question(df, abbreviation_to_field_name, corpus, dataset_id, original_dataset_id, spec_id, task_type)
    except Exception as e:
        print('Error generating question for task', task_type, e)
        continue
    try:
        fields_in_question = [question['c'], question['q1'], question['q2']]
        fields_in_df = list(df.columns)
        if (set(fields_in_question) != set(fields_in_df)):
            print('Mismatched fields', fields_in_df, fields_in_question)
    except Exception as e:
        print('Error matching fields', e)
        continue

    per_corpus_per_v_t[corpus] += 1

    valid += 1
    valid_per_corpus[corpus] += 1

    task_to_questions[task_type].append(question)
    valid_datasets = valid_datasets.append({
        'corpus': corpus,
        'locator': row['locator'],
        'dataset_id': dataset_id,
        'combination_number': combination_number,
        'column_metadata': column_metadata,
        'data': df.to_json(orient='records')  
    }, ignore_index=True)  # Save row for later

    if total % print_every == 0: print('[{}/{}] Corpus: {}, Dataset ID: {}'.format(valid, total, corpus, dataset_id))

    # Copy over dataset file
    output_file_name = join(data_dir, '{}.json'.format(dataset_id))
    if os.path.isfile(output_file_name):
        print('File already exists')
        print(corpus, combination_number, dataset_id)
    else:
        df.to_json(
            output_file_name,
            orient='records'
        )
        j = df.to_json(
            orient='records'
        )
        print(j)

    count_per_v_t[current_spec_id][current_task_id] += 1
    if count_per_v_t[current_spec_id][current_task_id] >= num_per_v_t:
        print('Spec ID:', current_spec_id)
        print('Task ID:', current_task_id)
        per_corpus_per_v_t = Counter()
        if current_task_id < len(task_types) - 1:  # Increment task
            current_task_id += 1
        else:
            if current_spec_id < len(raw_specs) - 1:
                current_spec_id += 1  # Increment spec
                current_task_id = 0
            else:
                break